# Lab 6: Vector Embeddings
So far, we've dealt primarily with real-valued input variables, like (x, y)-coordinates and pixel colors.
But lots of interesting input types, like text and location, are discrete.
Vector embeddings are the preferred way to deal with discrete inputs -- they're satisfying in theory, work well in practice, and have some really cool properties.

In [1]:
import numpy as np
import tensorflow as tf

# The problem with categorical variables
Imagine you want to train a model that takes a fixed-size set of words from a movie review (e.g. "this movie is great") and predicts the rating associated with that review.
It's not obvious how to take those words and turn them into something a neural network can use as input.

## Try 1: Integer encoding
A first try: just assign every word an integer!
Then "this movie is great" could turn into the vector [10, 32, 14, 9].
The problem with this approach is that it imposes bad priors on the input.
There are two big problems in particular:
 - The word "great" (represented by 9) isn't necessarily closer to the word "is" (represented by 14) than it is to the word "movie" (represented by 32), but it looks closer to the model because 9 is closer to 14 than it is to 32.
 - There isn't necessarily a transitive "ordering" on the words "great", "is", and "movie", but to the model it looks like there is because 9 < 14 < 32.
 
So just assigning every setting of the discrete variable a different integer doesn't work.

NOTE: sometimes a discrete variable does really have ordering and distance properties (for example, "number of transactions in the last day" might be an appropriate integer-encoded input for fraud detection).
But for anything categorical, integer encoding is a bad choice.

## Try 2: One-hot encoding
Another approach is one-hot encoding, which we've already seen for output categorical variables.
So, a given word would map to the vector [0, 0, ..., 1, 0, 0], with a 1 in a single space and a 0 everywhere else.
The big issues with this approach are:
 - When you have many categories (a model might consider ~2 million words for instance), every input is an extremely large, extremely sparse vector (a vector of length 2 million, with a single element of 1 and the rest 0). This can be very computationally and statistically inefficient.
 - The inputs all seem equidistant from each other, when in reality certain words _are_ similar to other words ("movie" should be similar to "movies", but with one-hot encoding they appear to be exactly as similar as "movie" and "is").
 
One-hot encoding is way better than integer encoding for categorical variables, but we can do better.

# Vector embeddings
Vector embeddings are a much nicer way to solve this problem.
For each possible value of the categorical variable, instead of a huge, sparse vector (e.g. [0, 0, ..., 0, 1, 0, ..., 0] of length ~2 million), we learn a smaller, dense vector (e.g. [0.1, -0.2, ..., 0.7] of length ~100-300).
This dense vector is called an "embedding vector" or just an "embedding."

To do this, we store a big lookup table of vectors, one for each value of the categorical variable.
When the input has a given value, the input to the next layer is the vector associated with that value.
Gradient descent with backpropagation works on these vectors, changing them incrementally at each step to improve loss just like model weights.
They're initialized randomly.

It's helpful to think of these vectors as defining a vector space, called "embedding space."

### Learned properties of embeddings
In practice, learned embeddings have some really interesting properties:
 - Models that use embeddings as inputs generalize much better than those that operate on e.g. n-grams.
 - Similar values (e.g. "movie" and "movies") are close together in embedding space (Euclidean distance or other). This indicates that absolute location in embedding space is meaningful, and tells us something about how embeddings help the model. You can also use these directly: consider training a model to put words into an embedding space and then comparing their distances to find similar words to an input. 
 - Directions in embedding space are generally meaningful too: see "embedding arithmetic" below.
 - Embeddings are extremely transferrable. A set of e.g. word embeddings learned on one task usually massively helps another task.
 - Embeddings can be learned unsupervised, and transferred to supervised learning tasks: see "learning embeddings unsupervised" below.
 
 As an example of how similar points have similar meanings, here's a list of words and the closest words to them in embedding space (learned unsupervised):
 ![close-word-embeddings](https://colah.github.io/posts/2014-07-NLP-RNNs-Representations/img/Colbert-WordTable2.png)
 (Source: [Collobert et al. (2011)](http://arxiv.org/pdf/1103.0398v1.pdf))
 
### Relationship with distributed representation
Given the above properties, it's easy to see why embeddings might help a model statistically.
But there's a deeper reason: embeddings are so powerful because they convert a symbolic representation, like one-hot encoding, into a **distributed representation** where a word is represented, in a kind of abstract way, by all of the components of its embedding at once.

If we want to learn a simple model (like an SVM) to operate over 5-grams from words alone, there are $(2M)^5$ possible inputs.
Treating every word as totally independent from every other word (like one-hot encoding does), it's impossible to deal with this with any reasonable amount of data.
Instead, embeddings compress the massive dimensionality of this space, discarding some information by moving similar words close together.

This lets the network reason by analogy, and intelligently combine words -- if the sentence "this movie is great" maps to a positive sentiment, and the sentence "the film is good" maps has a very similar set of embeddings, then it should map to a similar positive sentiment under the same model.
This is what really underlies the massive generalization advantages that distributed representations give us.

The key takeaway here is that _embeddings give the model a statistical advantage that's exponential in the number of words you're combining_.
This is really powerful!

NOTE: the concept of how distributed representation helps neural nets generalize goes way deeper than just word embeddings, but they're a particularly simple and powerful example of the idea 

### Equivalence to one-hot encoding
Mathematically, looking up the embedding of a word in a table is identical to multiplying that word's one-hot encoding by a weight matrix of size $(\text{number of words} \times \text{embedding size})$ -- multiplying a one-hot vector by a matrix just selects a single row from the matrix.

So, you can think of embeddings as equivalent to one-hot encoding your inputs, followed by a single dense layer that performs extreme dimensionality reduction and has no activation function.
From this perspective, the critical differences are:
 - Implementing embeddings as a lookup table is way more efficient than as a matrix multiplication.
 - Embeddings won't include a pointless bias vector.
 - Embeddings are interesting and transferrable in their own right, while the weights of a dense layer often are not.
 - Embeddings have many more interesting interpretations than "the weight matrix of the first dense layer."

### Embedding arithmetic
One fascinating property of learned embedding spaces is that directions in them are often interpretable, and you can do _meaningful arithmetic with these vectors!_ 
For instance, a famous example is as follows: if you take the embedding vector of the word "king", subtract the embedding of the word "man", and add the embedding of the word "woman", you wind up with a new vector in embedding space.
This point probably isn't exactly mapped to by any word, but _the closest point that is mapped to by a word is the embedding vector for "queen"!!!_ 

This means that between word pairs that vary in similar ways, the vector that points from one point in embedding space to another is just about the same!
![gender-vectors](https://colah.github.io/posts/2014-07-NLP-RNNs-Representations/img/Mikolov-GenderVecs.png)
(Source: [Mikolov et al. 2013](https://www.aclweb.org/anthology/N/N13/N13-1090.pdf))

Once we've trained vector embeddings for words or other data, we can use vector arithmetic on the embeddings to do all kinds of cool things, like automatically discovering analogies:
![analogies](https://colah.github.io/posts/2014-07-NLP-RNNs-Representations/img/Mikolov-AnalogyTable.png)
(Source: [Mikolov et al. 2013](https://www.aclweb.org/anthology/N/N13/N13-1090.pdf))

It also implies, again, that vector embeddings are really teaching our model something very rich about what the words mean.

#### Vector embeddings are super cool
For more about embeddings, embedding spaces, and representation learning, I highly recommend [this post on Chris Olah's blog](https://colah.github.io/posts/2014-07-NLP-RNNs-Representations/).
Seriously, this stuff is awesome.
Check it out.

# Learning embeddings unsupervised
It's extremely common to have huge databases of unlabeled text data (like Wikipedia), but very few labeled points.

One incredible property of vector embeddings is that they can be learned very effectively unsupervised, and then used in a supervised model.
All of the examples above of learned similarities and analogies come from embeddings learned unsupervised, so we can be sure that for text at least, unsupervised learning can result in really powerful word embeddings.

The key idea behind unsupervised learning of vector embeddings is to have a very simple (differentiable) "dummy" model try to solve a difficult task using only the embeddings as input.
For example, the famous Word2Vec model (see this week's assignment) tries to predict what word is missing from a sentence given the vector embeddings of the surrounding words, using only a linear model.
The model is nowhere near powerful enough to do this on its own (with e.g. one-hot encoded input), so for it to so well on the task, it needs the embeddings to convey a lot of information about the meanings of the input words.
So, throughout the course of training the model, backpropagation will result in good vector embeddings with the properties described above.

After that's done, you can throw away the dummy model -- it's just there to learn the word embeddings.
Then, you can use the word embeddings on any task that would have previously taken words as input to automagically improve performance on that task, since you'll have the benefit of word vectors trained on a massive unlabeled dataset even if you only have a small set of labeled examples.
As you train your model supervised, you can either "freeze" the word embeddings as they are (e.g. with `tf.stop_gradient`), or fine-tune them on your task by allowing backpropagation to keep modifying them. 

Even better, you can [download](https://fasttext.cc/docs/en/english-vectors.html) state-of-the-art pre-trained word embeddings from a number of top research groups, trained on massive datasets.
When you're writing a text model, starting with these word embeddings will almost always boost performance dramatically.

# Candidate sampling
A common situation when training large numbers of embeddings is that your model will need to predict one of very many classes.
When this is the case, using softmax output to compute the training loss is inefficient.
For every training example, computing the softmax loss involves computing a logit for each of the classes in a categorical output.
So, for a model that predicts a word, getting the loss for a single training example will involve computing ~2 million unnormalized probabilies, and this is of course true for each of the many steps of training.

Instead, it's more efficient to train using a procedure called "candidate sampling," which computes the Monte Carlo approximation to the loss.
There are a number of variants, which you can read about [here](https://www.tensorflow.org/extras/candidate_sampling.pdf) if you're interested.
The main idea behind each of these is that the true class is known, and instead of computing the softmax loss for all of the classes, you sample a few negative (incorrect) classes and compute the loss only for those.

This has an equivalent interpretation of training your model to distinguish the true target from a number of "noise" words.
The model is penalized for assigning probability to the noise words instead of the true target.

If you're using the model for something other than just training word embeddings, you should switch the output function from a candidate-sampling function to a full logistic or softmax function during inference -- candidate sampling is generally useful only during training, since it's less accurate than full softmax.

To use candidate sampling in TensorFlow, see the functions:
 - [`tf.nn.sampled_softmax_loss`](https://www.tensorflow.org/api_docs/python/tf/nn/sampled_softmax_loss) for sampled softmax output (given integer labels), or [`tf.contrib.nn.sampled_sparse_softmax_loss`](https://www.tensorflow.org/api_docs/python/tf/contrib/nn/sampled_sparse_softmax_loss) if your labels are one-hot encoded
 - [`tf.nn.nce_loss`](https://www.tensorflow.org/api_docs/python/tf/nn/nce_loss) for noise-contrastive estimation, a variant of sampled softmax
 
To use candidate sampling in Keras, you'll need to define a custom loss function.

# Embeddings in TensorFlow
The critical Operation when working with embeddings in TensorFlow is [`tf.nn.embedding_lookup`](https://www.tensorflow.org/api_docs/python/tf/nn/embedding_lookup), which takes a Tensor of embeddings and a Tensor of integers, and efficiently returns the embeddings at those integer indices.

I've given an example below:

In [18]:
# Create 4 embeddings of 3 numbers each
embedding_values = np.array([
    [0.1, 0.5, -0.7],
    [-0.6, 0.2, 0.3],
    [1.2, -0.9, 0.4],
    [0.7, 0.4, 0.9]
])

# In practice, this would be a tf.Variable, randomly initialized and learned
all_embeddings = tf.constant(embedding_values, name='all_embeddings')

# These would be e.g. the numbers of our words
word_ids = tf.placeholder(dtype=tf.int32, name='word_ids')

# Compute the embedding for each word
word_embeddings = tf.nn.embedding_lookup(all_embeddings, word_ids,
                                         name='word_embeddings')

# Print some embeddings
with tf.Session() as sess:
    print('Embedding for words 0 and 2:\n', 
          sess.run(word_embeddings, feed_dict={word_ids: [0, 2]}))
    print('Embedding for word 1:\n',
          sess.run(word_embeddings, feed_dict={word_ids: [1]}))

Embedding for words 0 and 2:
 [[ 0.1  0.5 -0.7]
 [ 1.2 -0.9  0.4]]
Embedding for word 1:
 [[-0.6  0.2  0.3]]


# Embeddings in Keras
To use vector embeddings in Keras, use [`keras.layers.Embedding`](https://keras.io/layers/embeddings/), which computes the embedding for every integer received as input.
To get the embeddings from the layer, you'll need to extract the layer's weights manually.

Here's an example of how to do both of those things:

In [50]:
from keras.models import Model
from keras.layers import Input, Embedding

inputs = Input(shape=())

# Get embedding layer separate from its output so we can extract weights
embedding_layer = Embedding(input_dim=8, output_dim=4)
embedding_outputs = embedding_layer(inputs)

# Build the model
model = Model(inputs=inputs, 
              outputs=embedding_outputs)
model.compile('SGD', loss='mean_squared_error')
model.summary()

# Extract all the embeddings
keras_embedding_values = embedding_layer.get_weights()
print('\nAll embeddings:\n', keras_embedding_values)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_14 (InputLayer)        (None,)                   0         
_________________________________________________________________
embedding_14 (Embedding)     (None, 4)                 32        
Total params: 32
Trainable params: 32
Non-trainable params: 0
_________________________________________________________________

All embeddings:
 [array([[-0.02984209,  0.03586355, -0.04011299, -0.03563318],
       [ 0.04278064, -0.04195632, -0.04200776, -0.04945996],
       [ 0.02155663, -0.00676335,  0.02107607, -0.03396394],
       [-0.00072262,  0.02625963,  0.0162635 , -0.02092161],
       [-0.02587697, -0.044058  , -0.03223451,  0.03688634],
       [-0.04710387, -0.04438034, -0.03857862,  0.01401922],
       [ 0.01136722, -0.04040575,  0.00580017,  0.04345243],
       [-0.00500871, -0.04298339, -0.033137  ,  0.02348597]],
      dtype=float32)]


In [47]:
print(embedding_layer.get_weights())

[array([[-0.03642311,  0.03057187,  0.0007695 , -0.00650785],
       [ 0.03990916,  0.04098651, -0.00464148,  0.03037584],
       [ 0.03350263,  0.02255045, -0.01729895, -0.01719505],
       [-0.01431742,  0.04087872, -0.04078948,  0.0392733 ],
       [ 0.01741653,  0.00063924,  0.04615687, -0.02768021],
       [ 0.04856498, -0.03933967,  0.03704797,  0.01389343],
       [-0.00645076,  0.03416571, -0.01932149,  0.02606802],
       [ 0.01315955,  0.0373205 ,  0.02260678,  0.02364657]],
      dtype=float32)]


# Visualize embeddings with Projector
One of the coolest features of TensorBoard is Projector, its tool for visualizing embeddings.
Try playing around with the online demo (of word embeddings) [here](https://projector.tensorflow.org/).
You can also use Projector in your local TensorBoard instance to view embeddings in your models with no extra code, just select the Tensor that holds your embeddings from the top-left dropdown.

To add metadata to embeddings (e.g. to see the words associated with embeddings), create a TSV file [as explained here](https://www.tensorflow.org/guide/embedding#metadata).

If you want to learn more about visualizing embeddings in TensorBoard, check out the official guide [here](https://www.tensorflow.org/guide/embedding).